In [170]:
import torch.nn.functional as F
import numpy
import torch

In [179]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

# Compute two different representation for each token.
# Each representation is a linear weighted combination for the
# 3 layers in ELMo (i.e., charcnn, the outputs of the two BiLSTM))
elmo = Elmo(options_file, weight_file, 1, scalar_mix_parameters = [1, 1, 1], dropout=0)

In [180]:
def similarity(x, y):
    return F.cosine_similarity(x, y, -1)

In [181]:
def sent2vec(s):
    character_ids = batch_to_ids([s])
    embeddings = elmo(character_ids)
    return embeddings['elmo_representations'][0][0][-1]

In [182]:
def count_score(x, pre_len, y, can):
    max_score = 0
    max_index = -1
    
    tmp = y.copy()
    tmp.append(can)
    y_vec = sent2vec(tmp)
    
    for i in range(pre_len+1, len(x)):
        x_vec = sent2vec(x[:i])
        score = similarity(x_vec, y_vec)
        if max_index < 0 or max_score < score:
            max_score = score
            max_index = i
    return max_index, max_score

In [183]:
def predict(x, pre_len, y, can_list):
    pre_lens = []
    scores = []
    for can in can_list:
        
        pre, score = count_score(x, pre_len, y, can)
        pre_lens.append(pre)
        scores.append(score)
    scores = torch.tensor(scores)
    qcm = F.softmax(scores, -1)
    return pre_lens, qcm

In [184]:
s1 = ['i', 'have', 'a', 'dream', '.']

In [185]:
predict(s1, 0, [], ['test', 'haha', 'like'])

([2, 1, 2], tensor([0.2884, 0.3789, 0.3327]))

In [178]:
scores

NameError: name 'scores' is not defined

In [106]:
a.append('b')

In [107]:
a

['a', 'b']